In [ ]:
# default_exp extend

# extend

> Extend an already existing `pd.DataFrame` with new data

In [ ]:
# export
import pathlib

import pandas as pd
import yaml

import dlsproc.structure
import dlsproc.bundle
import dlsproc.hier
import dlsproc.io
import dlsproc.assemble

In [ ]:
df_directory = pathlib.Path.cwd() / 'samples'
assert df_directory.exists()
print(df_directory)

/home/manu/dlsproc/samples


Input directory for the `pd.DataFrame` to be updated

In [ ]:
df_input_file = df_directory / '2018-2021_20samples.parquet'
assert df_input_file.exists()
print(df_input_file)

/home/manu/dlsproc/samples/2018-2021_20samples.parquet


The specific file is

In [ ]:
historical_df = pd.read_parquet(df_input_file)
historical_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                   

The directory for the zip file whose data is to appended to the already existing `pd.DataFrame`

In [ ]:
# zip_directory = pathlib.Path.cwd() / 'data' / 'agregados'
zip_directory = df_directory
assert zip_directory.exists()
print(zip_directory)

/home/manu/dlsproc/samples


The particular file

In [ ]:
# zip_file = zip_directory / 'PlataformasAgregadasSinMenores_202201.zip'
zip_file = zip_directory / 'PlataformasAgregadasSinMenores_202201_05-06.zip'
assert zip_file.exists()
print(zip_file)

/home/manu/dlsproc/samples/PlataformasAgregadasSinMenores_202201_05-06.zip


In [ ]:
# data is read from the above *zip* file, and concatenated into a single `pd.DataFrame`...
new_df = dlsproc.bundle.read_zip(zip_file, concatenate=True)

In [ ]:
# ...and re-structured with multiindexed columns
new_df = dlsproc.hier.flat_df_to_multiindexed_df(new_df)

Number of levels in the columns

In [ ]:
new_df.columns.nlevels

6

In [ ]:
# the same contract might show up more than once due to updates...but only the last one is kept
last_update_only_df = dlsproc.postprocess.keep_updates_only(new_df)

In [ ]:
# the same zip file also contains information (at the beginning) about deleted entries
deleted_series = dlsproc.bundle.read_deleted_zip(zip_file)

Duplicates are removed from the above `pd.Series`

In [ ]:
deduplicated_deleted_series = deleted_series.sort_values().groupby(deleted_series.index).tail(1)
deduplicated_deleted_series.head(2)

file name                                            id                                                                                 
PlataformasAgregadasSinMenores_20220105_030012.atom  https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1959074   2022-01-04 00:11:56.497000+01:00
                                                     https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1957921   2022-01-04 00:11:56.535000+01:00
Name: deleted_on, dtype: datetime64[ns, pytz.FixedOffset(60)]

In order to make it easy to update the `deleted_on` column in the existing `pd.DataFrame` using the data contained in the *deleted* `pd.Series` (i.e., to get *automatic 
alignment*), we reindex the dataframe in the same way as the series.

In [ ]:
reindexed_new_df = new_df.reset_index().set_index(['file name', 'id'])
reindexed_new_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20220105_030012.... https://contrataciondelestado.es/sindicacion/Pl...     0   
                                                   https://contrataciondelestado.es/sindicacion/Pl...     1   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20220105_030012.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2021039438; Órgano de Contratac...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20220105_030012.... https://contrataciondelestado.es/sindicacion/Pl...  L'objecte és la contractació del servei de bug...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Servei comunicacions postals de l'Ajuntament d...   

                                                                                                                               updated  \
                                                                                                                                         
                

What is the intersection between the entries in the new `pd.DataFrame` and the series with information about deletes.

In [ ]:
set(new_df['id']) & set(deduplicated_deleted_series.index.get_level_values(1))

set()

Old and new data are stacked together

In [ ]:
concatenated_df = dlsproc.assemble.stack(historical_df, new_df)
concatenated_df

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   
                                                   451    https://contrataciondelestado.es/sindicacion/P...   
                                                   450    https://contrataciondelestado.es/sindicacion/P...   
                                                   449    https://contrataciondelestado.es/sindicacion/P...   
..                                                                                                      ...   
PlataformasAgregadasSinMenores_20220106_030013.... 471    https://contrataciondelestado.es/sindicacion/P...   
                                                   472    https://contrataciondelestado.es/sindicacion/P...   
                                                   473    https://contrataciondelestado.es/sindicacion/P...   
                                                   474    https://contrataciondelestado.es/sindicacion/P...   
                                                   475    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   
                                                   451    Expediente: 1281/17, Entidad: Diputación Provi...   
                                                   450    Id licitación: VI/17/04-015; Órgano de Contrat...   
                                                   449    Id Licitación: PcPG/2017/194222, Órgano de Con...   
..                                                                                                      ...   
PlataformasAgregadasSinMenores_20220106_030013.... 471    Id licitación: 2021/57-12334; Órgano de Contra...   
                                                   472    Id licitación: 32/2021; Órgano de Contratación... 

In order to make it easy to update the `deleted_on` column in the existing `pd.DataFrame` using the data contained in the *deleted* `pd.Series` (i.e., to get *automatic 
alignment*), we reindex the dataframe in the same way as the series.

In [ ]:
res = concatenated_df.reset_index().set_index(['file name', 'id'])
res.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20180217_180137_... https://contrataciondelestado.es/sindicacion/Pl...   453   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   452   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20180217_180137_... https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                       

In [ ]:
deduplicated_deleted_series.index[0]

('PlataformasAgregadasSinMenores_20220105_030012.atom',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1959074')

In [ ]:
# res.loc[('PlataformasAgregadasSinMenores_20220105_030012.atom',
#  'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1959074')]

In order to see if something changes when updating column `deleted_on`

In [ ]:
bak_res = res.copy()

Update

In [ ]:
res['deleted_on'] = deduplicated_deleted_series

In [ ]:
bak_res.equals(res)

True